In [20]:
import random
import os
import pandas as pd
import numpy as np
import pickle as pkl
from collections import Counter
import itertools
from itertools import combinations

In [45]:
from BF_checkers import *
from RoF_checker import *
import BF_properties as BF_props
import BF_convertors as conv
from BF_generators import *

In [22]:
def IntsToBitsFast_lr (ints, size):
    '''
    Source: 
    -------
    https://stackoverflow.com/questions/
    
    Parameters:
    -----------
    ints: Numpy array of integers whose binary vectors are to be obtained
    size: The number of bits used to encode the integers provided in array ints 
    
    Output:
    -------
    Return a len(ints) X size matrix such that the MSB for each integer is the column 0 and LSB at column 'size-1'
    '''
    
    return (((ints[:,None] & (1 << np.arange(size, dtype = 'int64'))[::-1])) > 0).astype(int)

In [23]:
ints = np.array([4, 5])
size = 5
x = IntsToBitsFast_lr (ints, size)
print(x)

[[0 0 1 0 0]
 [0 0 1 0 1]]


# Computing the permutations of a BF using heap algorithm

In [24]:
def gen_heap_perms (k, f, k_list, tt):
    '''
    this function generates all the permutations of a BF expression
    for a given bias and given signs on inputs and is based on a code in
    https://stackoverflow.com/questions/29042819/heaps-algorithm-permutation-generator
    
    Paramters:
    ----------
    k: number of inputs
    f: BF as a string of bits
    k_list: list of the inputs
    tt: truth table input columns as a numpy matrix

    Output:
    --------
    returns the list of BFs as strings corresponding to all permuations of inputs
    '''
    
    if k == 1:
        yield f

    else:
        for i in range(k):
            
            for hp in gen_heap_perms(k-1, f, k_list, tt):
                yield hp

            if k & 1:
                k_list[0], k_list[k-1] = k_list[k-1], k_list[0]
                f = BF_props.bf(len(k_list), f).swap_cols((k_list[0], k_list[k-1]), tt)
               
            else:
                k_list[i], k_list[k-1] = k_list[k-1], k_list[i]
                f = BF_props.bf(len(k_list), f).swap_cols((k_list[i], k_list[k-1]), tt)

In [25]:
k1 = 3
f1 = '00000101'
k_list1 = list(range(k1, 0, -1))
tt1 = IntsToBitsFast_lr(np.arange(2**k1), k1)
L1 = list(gen_heap_perms (k1, f1, k_list1 ,tt1))
print(L1)

['00000101', '00010001', '00000011', '00010001', '00000011', '00000101']


# Z-parameter codes

In [26]:
## CODES FOR Z-PARAMETER COMPUTATION

## COMPUTING THE ARRAY NECESSARY TO GENERATE THE Z-PARAMETER
def left_z_count (k, f, k_inp):
    '''
    Parameters:
    -----------
    k: Number of inputs
    f: Boolean function as a string
    k_inp: The number of inputs (fixed)

    Output:
    -------
    This code returns an array (n_k vector defined in the text)
    0th element -> Cardinality of rows belong to CC-2^1 blocks
    1st element -> Cardinality of rows belong to CC-2^2 blocks
    2nd element -> Cardinality of rows belong to CC-2^3 blocks
    ......
    '''
    if k == 1:
        if (f == '01') or (f == '10'):
            return np.pad([2], [0,k_inp-k], mode='constant')

        else:
            return np.pad([0], [0,k_inp-k], mode='constant')
        
    else:
        p = 2**(k-1)
        f_upper, f_lower = f[:p], f[p:]
        a, b = set(f_upper), set(f_lower)
        if (len(a) == 1) & (len(b) == 1) & (a != b):
            count_array = np.array(k_inp*[0])
            count_array[k-1] = 2*p
            return count_array
        else:
            return left_z_count (k-1, f_upper, k_inp) + left_z_count (k-1, f_lower, k_inp)


## COMPUTING THE Z-LEFT VALUE OF THE BOOLEAN FUNCTION, BASED ON THE Z_COUNT ARRAY
def left_z_param_value (n_k_list): 
    '''
    Parameters:
    -----------
    n_k_list: The array generated from the left_z_count function

    Output:
    -------
    Z-left for the BF whose n_k_list is given as input
    '''
    inp_num = len(n_k_list)
    R_k = np.array(n_k_list)/(2**inp_num)
    z_left = R_k[0]
    if inp_num > 1:
        for i in range(1,inp_num):
            z_param_term = R_k[i]
            if z_param_term != 0:
                for j in range(0,i):
                    z_param_term *= (1-R_k[j])
                z_left += z_param_term
    return  z_left

## COMPUTING THE Z-PARAMETER VALUE OF A BOOLEAN FUNCTION BY OBTAINING THE MAXIMUM (Z_MAX), MINIMUM (Z_MIN), 
## AVERAGE (Z_AVE) OF ITS Z_LEFTs OVER ALL ITS PERMUTATIONS AND Z_MID (AVERAGE OF Z_MAX AND Z_MIN)

def z_param (k, f, tt, k_list):
    '''
    Parameters:
    -----------
    k: number of inputs
    f: The Boolean function as an integer
    tt: Truth table of the Boolean function
    k_list: List of k inputs 

    Output:
    -------
    4 possiblities of the Z-parameter of the Boolean function, namely:
    z_max, z_min, z_ave and z_mid
    ''' 
    
    f = bin(f)[2:].zfill(2**k)
    all_perms = set(list(gen_heap_perms (k, f, k_list, tt)))
    perm_z_values = []
    for func in all_perms:
        z_count = left_z_count (k, func, k)
        z_value = left_z_param_value(z_count)
        perm_z_values.append(z_value)
    z_min, z_ave, z_max = min(perm_z_values), np.mean(perm_z_values), max(perm_z_values)
    z_mid = (z_min + z_max)/2
    return  z_max, z_min, z_ave, z_mid

In [27]:
## Computation of Z_left without using a dictioanry
n_k_list_1 = left_z_count (5, '00110100000000000101010100001111',5)
left_z_param_value (n_k_list_1)

0.548828125

In [28]:
## Computation of Z-parameters for a Boolean function(Z_max, Z_min, Z_ave and Z_mid respectively)
k2 = 5
tt2 = IntsToBitsFast_lr(np.arange(2**k2), k2) ## function defined in heap_code
k_list2 = list(range(k2, 0, -1))
z_param(k2, 872437007, tt2, k_list2)

(0.578125, 0.390625, 0.46575520833333334, 0.484375)

# Code to generate BFs at each node that satisfy the attractor (fixed point) constraints

In [29]:
#This code generates the BFs which satisfy the attractor constraints at all
#nodes of the network

# The inputs to this code are:
# 1. an attractor file (contains all nodes and attractors): attractor.tsv
# 2. edgelist (derived from the network structure): edgelist.tsv
# Note: The BF to a node has a truth table like this:
# B | A | C | D(A,B,C)
# 0   0   0   0
# 0   0   1   0
# 0   1   0   0
# 0   1   1   0
# 1   0   0   0
# 1   0   1   0
# 1   1   0   0
# 1   1   1   0

# The order of the variables B, A, C from Left to Right are the same as the
#order of inputs to a node from top to bottom in the edgefile

# Using the attractors, the nodes where BFs are possible are constrianed 

##from BF_codes import *
##import sys
##sys.path.append('BF_codefile')

def model_data(fpath, dataset):
    '''
    path: a path to a file where all the data is present regarding the
    edges and attractor files
    returns information about the network and desired attractors
        -node_num: a dictionary where each gene is associated with an
        integer
        -inedges:  a dictionary where each gene is associated with its
        regulatory inputs
        -signs: a list of all the signs, in the order of the nodes
        -attr_as_matrix: the attractor specified as a matrix
    '''
    edges = pd.read_csv(fpath + fr'/edges_{dataset}.tsv', sep = '\t')
    bio_attrs = pd.read_csv(fpath + fr'/attractors_{dataset}.tsv', sep = '\t')
    node_num = {bio_attrs.node[i]:i for i in range(len(bio_attrs))}
    in_edges = dict()
    signs = dict()
    for node in bio_attrs.node:
        inter = edges.loc[edges["to"] == node, ["from", "sign"]]
        signs[node_num[node]] = ''.join(list(inter.sign))
        in_edges[node_num[node]] = [node_num[ele] for ele in inter['from']]
        if in_edges[node_num[node]] == []:
            in_edges[node_num[node]] = [node_num[node]]
            signs[node_num[node]] = 'a'
            print (f'Node {node} does not have any inputs. Self loop assigned!')
    attr_as_matrix = bio_attrs.drop('node', axis = 1).to_numpy()    
    return node_num, in_edges, attr_as_matrix, signs
    
def attr_constr_funcs(fpath, dataset):
    '''
    path: a path to a file where all the data is present regarding the
    edges and attractor files
    filename: name of the particular model under consideration
    returns a list of BFs in the order in 'node_num' which satisfy the
    attractor constraints
    NOTE: those rows of the truth table where a function output is replaced
    multiple times have to be excluded from the constraint
    '''
    attr_constr_BFs = dict()
    M = model_data(fpath, dataset)
    node_num, in_edges, attr_as_matrix = M[0], M[1], M[2]
    for node in node_num:
        i = node_num[node]
        x = np.array(attr_as_matrix[i], str)
        attr_constr_rows = np.zeros(len(attr_as_matrix.T), str)
        for in_edge in in_edges[i]:
            attr_constr_rows = np.char.add(attr_constr_rows, np.array(attr_as_matrix[in_edge], str))
        rows = np.array([int(ele,2) for ele in attr_constr_rows])
        rows, x = check_conflicting_constraints(rows, x) #Check for conflicting constraints for rows at the node under consideration
        tt = np.array (2**len(in_edges[i])*['x'])
        tt[rows] = x
        attr_constr_BFs[node_num[node]] = ''.join(tt)
    return attr_constr_BFs

def check_conflicting_constraints (rows, rows_output):
    '''
    rows: all the truth table rows which are constrainted by the attractor
    rows_output:  all the outputs of the rows of the truth table which
    are constrainted by the attractor
    returns rows and the row outputs with no conflicting constraints
    '''
    D, remove_indices = {}, []
    for element in set(rows):
        D[element] = [index for index, ele in enumerate(rows) if ele == element]
    for element in D:
        if len(set(rows_output[D[element]])) > 1:
            remove_indices += D[element]
            print ('There exists a conflict at row :', element)
    for index in sorted(remove_indices, reverse=True):
        rows = np.delete(rows, index)
        rows_output = np.delete(rows_output, index)
    return rows, rows_output

class constrainBF:
    '''
    #functionality
    given a BF constrained at some rows, this class gives the EFs, scEUFs, scRoFs or scNCFs that satisfy those 
    constraints
    '''
    def __init__(self, BF, sign):
        '''
        #arguments
        BF: Boolean function as a string of bits '11xx'; x can be 0 or 1
        sign: string of 'a' and 'i'; 'ai' where 'a' is activatory and 'i' is inhibitory
        '''
        self.BF = BF
        self.sign = sign
        self.constr_rows = [ind for ind, bit in enumerate(self.BF) if bit != 'x']
        self.fixed_bits = list(itemgetter(*self.constr_rows)(self.BF))

    def with_EF (self):
        k = len(self.sign)
        possible_EFs = []
        if 'x' not in self.BF:
            if check_if(k, self.BF).is_EF():
                possible_EFs += [int(self.BF,2)]
            else:
                print ('Given BF cannot be EF')
                
        else:
            for func in generate(k).all_BF():
                if check_if(k, func).is_EF():
                    if list(itemgetter(*self.constr_rows)(func)) == self.fixed_bits:
                        possible_EFs += [int(func,2)]
        return possible_EFs

    def with_scEUF (self):
        k = len(self.sign)
        uf_act_df = pd.read_csv(fr'../data/computational/UF_perms/all_UFs_k{k}.tsv', sep = '\t')
        euf_act_df = uf_act_df[uf_act_df['IEF']== False]
        inh_edges = [k-index for index, s in enumerate(self.sign) if s == 'i']                
        possible_scEUFs = []
        
        if 'x' not in self.BF:
            if check_if(k, self.BF).conforms_to_edge_signs(self.sign):
                possible_scEUFs += [int(self.BF,2)]
            else:
                print ('Given BF cannot be sc-EUF')
        else:
            for func in euf_act_df['all_UFs']:
                func = bf(k, str(func).zfill(2**k)).swap_rows(inh_edges)
                if list(itemgetter(*self.constr_rows)(func)) == self.fixed_bits:
                    possible_scEUFs += [int(func, 2)]
        return possible_scEUFs

    def with_scNCF (self):
        k = len(self.sign)
        possible_scNCFs = []
        inh_edges = [k-index for index, s in enumerate(self.sign) if s == 'i']

        if 'x' not in self.BF:
            if check_if(k, self.BF).conforms_to_edge_signs(self.sign):
                if check_if(k, self.BF).is_NCF():
                    possible_scNCFs += [int(self.BF, 2)]
            else:
                print ('Given BF cannot be scNCF')

        else:
            with open (fr'../data/computational/NCF_perms/all_perms_NCF{k}.txt', 'r') as file:
                for func in file:
                    func = bf(k, bin(int(func.strip('\n')))[2:].zfill(2**k)).swap_rows(inh_edges)
                    if list(itemgetter(*self.constr_rows)(func)) == self.fixed_bits:
                        possible_scNCFs += [int(func, 2)]
        
        return possible_scNCFs

    def with_scRoF (self):
        k = len(self.sign)
        possible_scrofs = []
        inh_edges = [k-index for index, s in enumerate(self.sign) if s == 'i']

        if 'x' not in self.BF:
            if check_if(k, self.BF).conforms_to_edge_signs(self.sign):
                if is_RoF(k, self.BF):
                    possible_scrofs += [int(self.BF, 2)]
            else:
                print ('Given BF cannot be scRoF')
            
        else:            
            with open (fr'../data/computational/RoF_perms/all_perms_RoF{k}.txt', 'r') as file:
                for func in file:
                    func = bf(k, bin(int(func.strip('\n')))[2:].zfill(2**k)).swap_rows(inh_edges)
                    if list(itemgetter(*self.constr_rows)(func)) == self.fixed_bits:
                        possible_scrofs += [int(func, 2)]
            
        return possible_scrofs

def constrain_all_nodes (BF_dict, signs, func_type_dict={}, all_type='scNCF'):
    '''
    Parameters:
    -----------
    BF_dict: is a dictionary with key as nodes and values as BFs that satisfy fixed point constraints
    signs: the signs of the inedges for each of the nodes
    func_type_dict: dictionary with key as nodes and values as the type of BF to be used to constrain that node
    all_type: type of BF to be used for constraining at each node
    
    Output:
    -------
    returns a dictionary with all the nodes and the `allowed' BFs satisfying fixed point and functional constraints
    '''
    constr_dict = {}

    if all_type != None:
        for node in BF_dict:
            func_type_dict[node] = all_type
            
    for node in BF_dict:
        if func_type_dict[node] == 'scEUF':
            constr_dict[node] = constrainBF(BF_dict[node], signs[node]).with_scEUF()
            
        elif func_type_dict[node] =='scRoF':
            constr_dict[node] = constrainBF(BF_dict[node], signs[node]).with_scRoF()    

        elif func_type_dict[node] == 'scNCF':
            constr_dict[node] = constrainBF(BF_dict[node], signs[node]).with_scNCF()    

    return constr_dict

# Implementation of the above code

In [30]:
## From the atrractor list and edgelist creates a list dictionary of node indexing (node_num), a dictionary of the 
## inputs to each node (inedges), the attractors as a matrix (attrs) and a dictionary consisting of the signs of 
## the inputs with the same order of the inputs in inedges(signs).
filename = 'buylla'
path = fr'../data/data_10_Boolean_models/{filename}'
node_num, inedges, attrs, signs = model_data(path, filename)
print('node_num:',node_num)
print('inedges:',inedges)
print('attrs:',attrs)
print('signs:',signs)

node_num: {'SCR': 0, 'PLT': 1, 'ARF': 2, 'AUXIAA': 3, 'AUXIN': 4, 'SHR': 5, 'JKD': 6, 'MGP': 7, 'WOX5': 8}
inedges: {0: [5, 6, 7, 0], 1: [2], 2: [3], 3: [4], 4: [4], 5: [5], 6: [5, 0], 7: [5, 8, 0], 8: [5, 2, 8, 0, 7]}
attrs: [[1 0 1 0]
 [1 1 1 1]
 [1 1 1 1]
 [0 0 0 0]
 [1 1 1 1]
 [1 1 1 0]
 [1 0 1 0]
 [0 0 1 0]
 [1 0 0 0]]
signs: {0: 'aaia', 1: 'a', 2: 'i', 3: 'i', 4: 'a', 5: 'a', 6: 'aa', 7: 'aia', 8: 'aaaai'}


In [31]:
## Returns a dictionary of BFs in the order in 'node_num' which satisfy the attractor constraints (constrained rows 
## are replaced by 0 or 1 according to the attractors whereas outputs are kept 'x' where rows are not constrained by 
## the attractors)
BF_dict = attr_constr_funcs(path, filename)
print(BF_dict)

{0: '0xxxxxxx0xxxx1x1', 1: 'x1', 2: '1x', 3: 'x0', 4: 'x1', 5: '01', 6: '0x01', 7: '0xxx01x0', 8: 'xxxxxxxx0xxxxxxxxxxxxxxx0xx0xx1x'}


In [32]:
## we can get all the functions (integer form) that satisfy the attractor constraints and fall under some specific
## for any node in the network. For example, here we are computing the sign-conforming NCF functions at the 0'th node
## that satisfy the attractor constraint as well.
L = constrainBF(BF_dict[0], signs[0]).with_scNCF()
print(L)

[1029, 69, 13, 1109, 1293, 20319, 79, 1349, 23903, 93, 1039, 3085, 17477, 3343, 3919, 17749, 21853]


In [33]:
## Returns the dictioary where the keys are the node indices as in node_num and the values are the list of all
## functions of the type mentioned (here 'scNCF') that satisfy the attractor constraints
func_dict = constrain_all_nodes (BF_dict, signs, func_type_dict={}, all_type='scNCF')
print(func_dict)

{0: [1029, 69, 13, 1109, 1293, 20319, 79, 1349, 23903, 93, 1039, 3085, 17477, 3343, 3919, 17749, 21853], 1: [1], 2: [2], 3: [2], 4: [1], 5: [1], 6: [1], 7: [4], 8: [2, 42, 11, 131082, 35, 522, 131586, 131075, 546, 131106, 515, 196643, 197123, 131843, 2228770, 131087, 2228266, 139810, 33686026, 33686050, 8746, 133642, 47, 2602, 33686019, 655371, 8739, 196619, 2228259, 527, 655402, 655882, 803, 2571, 779, 2831, 34212362, 35791394, 720911, 33751811, 983567, 168430122, 50529035, 3887, 983087, 134927, 572662307, 168430091, 50529059, 572662314, 34540047, 573188650, 51055371, 168430351, 985871, 52634403, 168495883, 50531087, 168495631, 724751, 34538255, 34213647, 170535466, 572728099, 51053327, 33754895, 185273103, 254750511, 51056399, 51317519, 168496911, 34541327, 168758031, 185536271, 252645167]}


# Random EUF generator

In [34]:
def get_all_forward_ele (vertices, all_vertex_neibs, ham_dist):
    '''
    Parameters:
    -----------
    vertices: the set of vertices whose influence is to be propagated
    all_vertex_neibs: the one Hamming neighbors of all the vertices provided as a dictionary
    ham_dist: the Hamming distance from 'vertices' to the vertex 2**k - 1
    
    Output:
    -------
    returns the list of all vertices 'ahead' of the vertices that lie in the same plane
    '''
    
    if ham_dist == 0:
        return list(set(vertices))

    else:
        new_vertices = []
        for vertex in vertices:
            for neib in all_vertex_neibs[vertex]:
                if neib > vertex:
                    new_vertices += [neib]
                    
        return list(set(vertices + get_all_forward_ele (new_vertices, all_vertex_neibs, ham_dist-1)))

def get_all_backward_ele (vertices, all_vertex_neibs, ham_dist):
    '''
    vertices: the set of vertices whose influence is to be propagated
    all_vertex_neibs: the one Hamming neighbors of all the vertices provided as a dictionary
    ham_dist: the Hamming distance from 'vertices' to the vertex 0
    
    Output:
    -------
    returns the list of all vertices 'ahead' of the vertices that lie in the same plane
    '''
    
    if ham_dist == 0:
        return list(set(vertices))

    else:
        new_vertices = []
        for vertex in vertices:
            for neib in all_vertex_neibs[vertex]:
                if neib < vertex:
                    new_vertices += [neib]
                    
        return list(set(vertices + get_all_backward_ele (new_vertices, all_vertex_neibs, ham_dist-1)))

def propogation_function (k, f, ind):
    '''
    it propogates the influence of the color of a vertex so as to satisfy the unateness property of the BF
    k: number of inputs
    f: BF as a truth table of 1s, 0s and xs
    ind: index to be propogated
    This assumes that all the regulators are activators
    '''
    all_vertex_neibs = bf(k, f).inps_neibs()
    list_f = list(f)

    if list_f[ind] == '0':
        ham_dist_z = bin(0^ind).count('1')
        z_neibs_fix = get_all_backward_ele ([ind], all_vertex_neibs, ham_dist_z)
        for z in z_neibs_fix:
            list_f[z] = '0'

    else:
        ham_dist_o = bin(2**k - 1^ind).count('1')
        o_neibs_fix = get_all_forward_ele ([ind], all_vertex_neibs, ham_dist_o)
        for o in o_neibs_fix:
            list_f[o] = '1'

    return ''.join(list_f)

def generate_random_UF(k, f):
    '''
    Parameters:
    -----------
    k: Number of inputs
    f: BF as string of size 2^k only with 'x' 
    E.g if k=2, then f = 'xxxx'
    if k=3, then f = 'xxxxxxxx'
    
    Output:
    -------
    A UF with all inputs activatory. The UF is represented as a string of bits. 
    
    '''
    x_ind = []
    for ind, bit in enumerate(f):
        if bit == 'x':
            x_ind += [ind]

    rand_ind = random.choice(x_ind)
    list_f = list(f)
    list_f[rand_ind] = str(np.random.randint(0,2))
    new_f = ''.join(list_f)
    f = propogation_function (k, new_f, rand_ind)
    list_f = list(f)
    
    if 'x' not in f:
        return f

    else:
        return generate_random_UF (k, ''.join(list_f))
    
def obtained_distr_UFs(k, samp):
    '''
    k: number of inputs
    samp: sample size
    '''
    distr = []
    for i in range(samp):
        f = generate_random_UF (k, 2**k *'x')
        distr += [int(f,2)]
    j = dict(Counter(distr))
    return j

def obtained_distr_EUFs(k, samp):
    '''
    k: number of inputs
    samp: sample size
    '''
    distr = []
    while len(distr) != samp:
        f = generate_random_UF (k, 2**k *'x')
        if check_if(k,f).is_EF():
            distr += [int(f,2)]
    j = dict(Counter(distr))
    return j

In [35]:
# Generates a random UF
print('A random UF:',generate_random_UF(2, 'xxxx'))
# Gives a dictionary of frequency distribution of UFs for a specific input number and sample size
print('Distribution of sampled random UFs:', obtained_distr_UFs(2, 100))
# Gives a dictionary of frequency distribution of EUFs for a specific input number and sample size
print('Distribution of sampled random EUFs:',obtained_distr_EUFs(2, 100))

A random UF: 0101
Distribution of sampled random UFs: {0: 23, 5: 18, 3: 13, 15: 22, 7: 11, 1: 13}
Distribution of sampled random EUFs: {1: 60, 7: 40}


# Nodewise sampling code

In [36]:
## This code returns required number of sampled functions for any node. Each function takes 4 inputs (except EF).
## k: number of inputs to the node , f: an attractor constrained function, M: the number of functions to be sampled
## sign: sign combination of the inputs according to the order of inputs in inedges dictionary.
## Here two types of sampling have been considered (exact and approximated) where exact sampling is uniform sampling
## for EFs and scEUFs. The type of sampling depends upon the number of inputs according to the given tsv file 
##'samp_type_info' provided at in the 'computational' folder inside 'data' folder. For scNCFs and scRoFs only exact
## sampling has been used.

def exact_samp_EF (k, f, M):
    '''
    k: number of inputs
    f: Boolean function with fixed point constraints
    M: number of functions to sample
    '''
    dummy_sign = k*'a'
    ef_list = constrainBF(f, dummy_sign).with_EF()    
    sampled_efs = random.choices(ef_list, k=M)
    return sampled_efs

def approx_samp_EF (k, f, M):
    '''
    k: number of inputs
    f: Boolean function with fixed point constraints
    M: number of functions to sample
    '''
    x_pos = [ind for ind, bit in enumerate(f) if bit == 'x']
    list_f = list(f)
    samp_EFs = []
    while (M != 0):
        for ind in x_pos:
            list_f[ind] = str(np.random.randint(0,2))
        if check_if(k,''.join(list_f)).is_EF() == True:
            samp_EFs += [int(''.join(list_f),2)]
            M -= 1
    return samp_EFs

def exact_samp_scEUF (k, f, sign, M):
    '''
    k: number of inputs
    f: Boolean function with fixed point constraints
    sign: string of length k whose elements are 'a' or 'i'. e.g. 'aiiiai'
    M: number of functions to sample
    '''
    sceuf_list = constrainBF(f, sign).with_scEUF()    

    sampled_sceuf = random.choices(sceuf_list, k=M)
    return sampled_sceuf

def approx_samp_scEUF (k, f, sign, M):
    '''
    k: number of inputs
    f: Boolean function with fixed point constraints
    sign: string of length k whose elements are 'a' or 'i'. e.g. 'aiiiai'
    M: number of functions to sample
    '''
    fps_pos = {ind: bit for ind, bit in enumerate(f) if bit != 'x'}
    inh_edges = [k-index for index, s in enumerate(sign) if s == 'i']

    # flip the signs of the inhibitory edges and make BF activatory
    flip_f = bf(k,f).swap_rows(inh_edges)
    new_fps_pos = {ind: bit for ind, bit in enumerate(flip_f) if bit != 'x'}
    for ind in list(new_fps_pos.keys()):
        flip_f = propogation_function (k, flip_f, ind)

    samp_eufs= []
    while M != 0:
        euf = False
        while euf == False:
            rand_uf = generate_random_UF(k, flip_f)
            if check_if(k, rand_uf).is_EF():
                euf = True
            else:
                euf = False
                
        # flip the signs of the inhibitory edges that were made activatory
        rand_euf = bf(k, rand_uf).swap_rows(inh_edges)
        samp_eufs += [int(rand_euf,2)]
        M -= 1
        
    return samp_eufs

def exact_samp_scRoF (k, f, sign, M):
    '''
    k: number of inputs
    f: Boolean function with fixed point constraints
    sign: string of length k whose elements are 'a' or 'i'. e.g. 'aiiiai'
    M: number of functions to sample
    '''
    scrof_list = constrainBF(f, sign).with_scRoF()
    
    sampled_scrof = random.choices(scrof_list, k=M)
    return sampled_scrof

def exact_samp_scNCF (k, f, sign, M):
    '''
    k: number of inputs
    f: Boolean function with fixed point constraints
    sign: string of length k whose elements are 'a' or 'i'. e.g. 'aiiiai'
    M: number of functions to sample
    '''
    scncf_list = constrainBF(f, sign).with_scNCF()    

    sampled_scncf = random.choices(scncf_list, k=M)
    return sampled_scncf

# Implementation of the above code

In [37]:
## One can generate required number of sampled functions in the following manner.
approx_EF = approx_samp_EF (len(inedges[8]), BF_dict[8], 10)
print('10 sampled EF:', approx_EF)
exact_scRoF = exact_samp_scRoF (len(inedges[8]), BF_dict[8], signs[8], 10)
print('10 sampled RoFs:', exact_scRoF)

10 sampled EF: [2186460459, 3439361570, 3662124046, 621504846, 3576418638, 3848646478, 3361020206, 1667566607, 2201425451, 2189215759]
10 sampled RoFs: [34538255, 52634403, 139810, 252645167, 33686050, 50531087, 983567, 131843, 655371, 199179]


# Nodewise enumeration of number of BFs

In [51]:
#from mod_sel_constrBF_final import *

def analysis_type (model_name):
    '''
    model name: A string correspond to one of the 10 reconstructed Boolean model networks (e.g. 'buylla', 't_cell',...) 
    '''
    path = fr'../data/data_10_Boolean_models/{model_name}'
    node_num, inedges, attrs, signs = model_data(path, model_name)

    samp_info = pd.read_csv('../data/computational/samp_type_info.tsv', sep = '\t')
    num_node = {node_num[node]:node for node in node_num}
    data_dict = {'Node_num': [], 'Node_name': [], 'k':[], 'EF':[], 'scEUF':[], 'scRoF':[], 'scNCF':[]}
    for num in inedges:
        k = len(inedges[num])
        data_dict['Node_num'] += [num]
        data_dict['Node_name'] += [num_node[num]]
        data_dict['k'] += [k]
        data_dict['EF'] += list(samp_info[samp_info['k'] == k]['EF'])
        data_dict['scEUF'] += list(samp_info[samp_info['k'] == k]['scEUF'])
        data_dict['scRoF'] += list(samp_info[samp_info['k'] == k]['scRoF'])
        data_dict['scNCF'] += list(samp_info[samp_info['k'] == k]['scNCF'])
    df = pd.DataFrame.from_dict(data_dict)
    return df

all_models = ['buylla', 't_cell', 'myeloid', 'cortical', 'drug', 'cardiac', 'sullivan', 'emt_senescent', 'corrales', 'gonadal']

for model_name in all_models:
    df = analysis_type(model_name)
    df.to_csv(fr'../data/nodewise_enum_BF/Sampling_type_information/{model_name}_samp_type_info.tsv', sep = '\t', index = False)

Node IFNb does not have any inputs. Self loop assigned!
Node TCR does not have any inputs. Self loop assigned!
Node IL18 does not have any inputs. Self loop assigned!
Node IL12 does not have any inputs. Self loop assigned!


In [49]:
import pickle as pkl

def model_statistics (model_name):
    path = fr'../data/data_10_Boolean_models/{model_name}'
    node_num, inedges, attrs, signs = model_data(path, model_name)
    samp_info = pd.read_csv(f'../data/nodewise_enum_BF/Sampling_type_information/{model_name}_samp_type_info.tsv', sep = '\t')
    BF_list = attr_constr_funcs(path, model_name)

    data_dict = {'Node_num': list(samp_info['Node_num']),'EF':[], 'scEUF':[], 'scRoF':[], 'scNCF':[]}

    for f_type in ['EF', 'scEUF', 'scRoF', 'scNCF']:
        if f_type == 'EF':
            for num in inedges:
                if samp_info[samp_info['Node_num']== num][f_type][num] == 'exact':
                    data_dict[f_type] += [len(constrainBF(BF_list[num], signs[num]).with_EF())]
                else:
                    data_dict[f_type] += ['-']

        elif f_type == 'scEUF':
            for num in inedges:
                if samp_info[samp_info['Node_num']== num][f_type][num] == 'exact':
                    data_dict[f_type] += [len(constrainBF(BF_list[num], signs[num]).with_scEUF())]
                else:
                    data_dict[f_type] += ['-']

        elif f_type == 'scRoF':
            for num in inedges:
                if samp_info[samp_info['Node_num']== num][f_type][num] == 'exact':
                    data_dict[f_type] += [len(constrainBF(BF_list[num], signs[num]).with_scRoF())]
                else:
                    data_dict[f_type] += ['-']

        elif f_type == 'scNCF':
            for num in inedges:
                if samp_info[samp_info['Node_num']== num][f_type][num] == 'exact':
                    data_dict[f_type] += [len(constrainBF(BF_list[num], signs[num]).with_scNCF())]
                else:
                    data_dict[f_type] += ['-']
    
    df = pd.DataFrame.from_dict(data_dict)
    return df

all_models = ['buylla', 't_cell', 'myeloid', 'cortical', 'drug', 'cardiac', 'sullivan', 'emt_senescent', 'corrales', 'gonadal']

# --------------------------------------------------------------------------------------------------------------------------
# CODE TO OBTAIN THE NUMBER OF BFs AT EACH NODE THAT SATISFY THE FIXED POINT CONSTRAINTS AND CREATE A TABLE OF THE SAME
# --------------------------------------------------------------------------------------------------------------------------
for model_name in all_models:    
   df = model_statistics (model_name)

   # Generate the total number of BFs possible for each ensemble and also
   # indicate whether exhaustive computations can be carried out or not
   tot_num, samp_row = [], []

   for f_type in ['EF', 'scEUF', 'scRoF', 'scNCF']:
       f_type_list = list(df[f_type])
       if '-' in f_type_list:
           tot_num += ['-']
           samp_row += ['True']
       else:
            num_mod = 1
            for ele in f_type_list:
                num_mod *= ele
            num = np.prod(f_type_list)
            tot_num += [num_mod]
            if num_mod <= 1000000:
                samp_row += ['False']
            else:
                samp_row += ['True']

   df.loc[len(df)] = ['Total'] + tot_num
   df.loc[len(df)] = ['Sampled'] + samp_row
   df.to_csv(f'../data/nodewise_enum_BF/model_statistics/{model_name}_model_stats.tsv', sep='\t', index= False)
   print (f'{model_name} is completed !')

buylla is completed !
Node IFNb does not have any inputs. Self loop assigned!
Node TCR does not have any inputs. Self loop assigned!
Node IL18 does not have any inputs. Self loop assigned!
Node IL12 does not have any inputs. Self loop assigned!
Node IFNb does not have any inputs. Self loop assigned!
Node TCR does not have any inputs. Self loop assigned!
Node IL18 does not have any inputs. Self loop assigned!
Node IL12 does not have any inputs. Self loop assigned!
t_cell is completed !
myeloid is completed !
cortical is completed !
drug is completed !
cardiac is completed !
sullivan is completed !
emt_senescent is completed !
corrales is completed !
gonadal is completed !


In [53]:
all_models = ['buylla', 't_cell', 'myeloid', 'cortical', 'drug', 'cardiac', 'sullivan', 'emt_senescent', 'corrales', 'gonadal']

for model_name in all_models:
    df_samp_info = pd.read_csv(f'../data/nodewise_enum_BF/Sampling_type_information/{model_name}_samp_type_info.tsv', sep = '\t')
    df_model_stats= pd.read_csv(f'../data/nodewise_enum_BF/model_statistics/{model_name}_model_stats.tsv', sep='\t')
    node_name, num_inps = list(df_samp_info['Node_name']), list(df_samp_info['k'])
    
    df_model_stats['Node name'] = node_name + ['' , '']
    df_model_stats['k'] = num_inps + ['', '']
    df_model_stats['Sr No.'] = list(range(1, len(node_name)+1)) + ['Total', 'Sampled']
    
    df_temp = df_model_stats[['Sr No.', 'Node name', 'k', 'EF', 'scEUF', 'scRoF', 'scNCF']]
    df_final = df_temp.rename(columns={'scEUF': 'EUF', 'scRoF': 'RoF', 'scNCF': 'NCF'}) ## To be consistent with
    # the manuscript we have dropped the 'sc'.
    df_final.to_csv (f'../data/nodewise_enum_BF/model_all_info/{model_name}_all_details.tsv', sep ='\t', index = False)

# Generating the boolnet files from pkl file

In [52]:
model_name = 'buylla'
f_type= 'EF'
with open (fr'../data/data_10_Boolean_models/{model_name}/1e6_sampled_models/{f_type}_sample.pkl', 'rb') as file:
    models = pkl.load(file)
test_models = models[:20]
path = fr'../data/data_10_Boolean_models/{model_name}'
filename = fr'{model_name}'
node_num, inedges, attrs, signs = model_data(path, filename)
tot_func = len(test_models)
output_path = fr'../data/data_10_Boolean_models/{model_name}/{f_type}_bnet_models/'
os.makedirs(output_path, exist_ok=True)
for i in range(tot_func):
    fname = output_path + f'model_{i}.bnet'
    BF_int_list = test_models[i]
    conv.ints_to_BNet(fname, node_num, inedges, BF_int_list, simplify=False)